In [1]:
import numpy as np
import utils
from PIL import Image as pil_image

def show(im):
    # Plots numpy array as pil image
    return pil_image.fromarray(im.astype(np.uint8))

def gauss_pdf(sigma, x, y):
    return np.exp(-(x ** 2 + y ** 2) / (2 * sigma ** 2)) / (2 * np.pi * sigma ** 2)

def gauss_filter(sigma, size=5):
    kernel = np.zeros((size, size))

    for i in range(size):
        for j in range(size):
            kernel[i, j] = gauss_pdf(sigma, i - size // 2, j - size // 2)

    return kernel

def sobel_filters(im):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    channels = im.shape[2]
    Ix = np.sum(utils.convolve_image(im, Kx), axis=2) / channels
    Iy = np.sum(utils.convolve_image(im, Ky), axis=2) / channels

    G = np.hypot(Ix, Iy)

    magnitude = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)

    return (magnitude, theta)

def round_angle(angle):
    # Angle is rounded to one of four angles (0, PI/4, PI/2 and 3PI/4)
    angle = (angle + np.pi) % np.pi
    if angle < np.pi / 8 or angle > 7 * np.pi / 8:
        return 0
    if angle < np.pi / 8 + np.pi / 4:
        return np.pi / 4
    if angle < np.pi / 8 + np.pi / 2:
        return np.pi / 2
    return 3 * np.pi / 4


def non_max_suppression(magnitude, theta):
    height, width = magnitude.shape
    result = np.zeros(magnitude.shape)

    for i in range(1, width - 1):
        for j in range(1, height - 1):
            angle = round_angle(theta[i, j])
            m = magnitude[i, j]

            if angle == 0:
                # Check east-west directions neighbours magnitude
                m1 = magnitude[i, j + 1]
                m2 = magnitude[i, j - 1]

            if angle == np.pi / 4:
                # Check north east and south west directions
                m1 = magnitude[i + 1, j - 1]
                m2 = magnitude[i - 1, j + 1]
            elif angle == np.pi / 2:
                # Check north and south directions
                m1 = magnitude[i + 1, j]
                m2 = magnitude[i - 1, j]
            else:
                # Check north west and south-east
                m1 = magnitude[i - 1, j - 1]
                m2 = magnitude[i + 1, j + 1]

            if m > m1 and m > m2:
                result[i, j] = m
            else:
                # Magnitude is suppressed
                result[i, j] = 0

    return result

def hysteresis(im, strong, weak):
    result = np.zeros(im.shape)

    for i in range(1, im.shape[0] - 1):
        for j in range(1, im.shape[1] - 1):
            if im[i, j] == strong:
                result[i, j] = strong
            if im[i, j] == weak:
                # Check if some of 8-connected neighbours is strong
                neighbours = np.array(
                    [
                        im[k, l]
                        for k in range(i - 1, i + 2)
                        for l in range(j - 1, j + 2)
                        if not (k == i and l == j)
                    ]
                )
                if np.any(neighbours == strong):
                    result[i, j] = strong
    return result

def double_threshold(im, low_ratio=0.05, high_ratio=0.1):
    result = np.zeros(im.shape).astype(np.uint8)

    high_threshold = im.max() * high_ratio
    low_threshold = high_threshold * low_ratio

    high_i, high_j = np.where(im >= high_threshold)
    low_i, low_j = np.where((im >= low_threshold) & (im < high_threshold))

    strong = 255
    weak = 30

    result[high_i, high_j] = strong
    result[low_i, low_j] = weak

    return result, strong, weak

# 전체 사진 불러오는 함수
def load_images(folder_path,file_names,img_size_shape):
    images = []                                          # 불러올 사진을 담을 리스트
    
    for i in range(len(file_names)):
        path = os.path.join(folder_path,file_names[i])   # 사진경로 생성
        img = Image.open(path).resize(img_size_shape)    # 이미지 오픈 후 리사이징
        img = img.convert('RGB')                         # PNG -> RGB
        images.append(np.array(img))                     # numpy배열로 변환 후 리스트에 추가
   
    return images

In [2]:
from PIL import Image
from matplotlib import pyplot as plt

f = gauss_filter(1.4)

for i in range(1, 3566, 1):
    try:
        image = Image.open('./추려봄/하트/heart_' +  str(i) + '.jpg')
        image = image.resize((256, 256))
        image_numpy = np.array(image)
        image_smoothed = utils.convolve_image(image_numpy, f)
        magnitude, theta = sobel_filters(image_smoothed)
        suppressed_magnitude = non_max_suppression(magnitude, theta)
        magnitude_thresholded, strong, weak = double_threshold(suppressed_magnitude)
        edges = hysteresis(magnitude_thresholded, strong, weak)
        plt.imsave('./엣지/하트/heart_' + str(i) + '.jpg', edges)
    except:
        pass